## Creating an Initial Population

Now we can create an initial population, by first defining the population size and then selecting from the list of recipes.




In [20]:
import json
import pprint
import random
import math
from collections import defaultdict
import copy
import pandas as pd
import statistics as stat
from sugarcube import Mass, Volume, Liquids, Butter, Sugar, Flour, Pastes, Solids, Powder, Bindings, Yeast, Salt, Icecream, Jam, Biscuits, Nori, Beans

In [21]:
with open('data/mochi.json', 'r') as file:
    data = json.load(file)
recipes = data['recipes']

So here we give each ingredient the rating of its recipe.
If an ingredient is used in multiple recipes, we'll take the mean of all the ratings for this ingredient.

In [53]:
all_ingredients = []
for i, recipe in enumerate(recipes):
  all_ingredients.extend(recipe['ingredients'])
pprint.PrettyPrinter(width=150).pprint(all_ingredients)

[{'amount': 283.5, 'ingredient': 'fresh strawberries', 'rating': 4.71, 'unit': 'g'},
 {'amount': 210.0, 'ingredient': 'mochiko flour', 'rating': 4.71, 'unit': 'g'},
 {'amount': 210.0, 'ingredient': 'sweet rice flour', 'rating': 4.71, 'unit': 'g'},
 {'amount': 210.0, 'ingredient': 'glutinous rice flour', 'rating': 4.71, 'unit': 'g'},
 {'amount': 45.0, 'ingredient': 'sugar of choice', 'rating': 4.71, 'unit': 'g'},
 {'amount': 52.5, 'ingredient': 'cornstarch', 'rating': 4.71, 'unit': 'g'},
 {'amount': 52.5, 'ingredient': 'potato starch', 'rating': 4.71, 'unit': 'g'},
 {'amount': 52.5, 'ingredient': 'tapioca starch', 'rating': 4.71, 'unit': 'g'},
 {'amount': 288.0, 'ingredient': 'sweetened white bean paste (shiro an)', 'rating': 4.71, 'unit': 'g'},
 {'amount': 140.0, 'ingredient': 'mochiko flour', 'rating': 5.0, 'unit': 'g'},
 {'amount': 140.0, 'ingredient': 'sweet rice flour', 'rating': 5.0, 'unit': 'g'},
 {'amount': 140.0, 'ingredient': 'glutinous rice flour', 'rating': 5.0, 'unit': 'g'}

In [60]:
all_names = set()
for ingredient in all_ingredients:
    all_names.add(ingredient['ingredient'])


In [51]:
population_size = 20
population = random.choices(recipes, k=population_size)
pprint.PrettyPrinter(indent=4, depth=2).pprint(population)

[   {'ingredients': [...], 'name': 'Pandan Donuts', 'rating': 3.8},
    {'ingredients': [...], 'name': 'Biscoff Baked Mochi Donuts', 'rating': 3.0},
    {'ingredients': [...], 'name': 'Purple Sweet Potato Mochi', 'rating': 4.86},
    {'ingredients': [...], 'name': 'Tofu', 'rating': 4.56},
    {'ingredients': [...], 'name': 'Purple Sweet Potato Mochi', 'rating': 4.86},
    {'ingredients': [...], 'name': 'Mugwort Mochi', 'rating': 4.8},
    {'ingredients': [...], 'name': 'Black Sesame', 'rating': 5.0},
    {'ingredients': [...], 'name': 'Ube Mochi Muffins', 'rating': 3.5},
    {'ingredients': [...], 'name': 'Purple Sweet Potato Mochi', 'rating': 4.86},
    {'ingredients': [...], 'name': 'Biscoff Baked Mochi Donuts', 'rating': 3.0},
    {'ingredients': [...], 'name': 'Matcha Baked Mochi Donuts', 'rating': 3.3},
    {'ingredients': [...], 'name': 'Chocolate Mochi', 'rating': 5.0},
    {'ingredients': [...], 'name': 'Pumpkin', 'rating': 4.8},
    {'ingredients': [...], 'name': 'Chocolate Mo

## Evaluating Recipes (Fitness Function)

The following function defines how individuals are evaluated:
We chose to evaluate the recipe by calculating the mean of the rating of its ingredients. So, we sum up the rating for each ingredient, and then divide by the total amount of ingredients.

In [61]:
# lists of constraints on the fitness function
presence_egg = ["egg"]
presence_gluten = ["bread flour", "all purpose flour", "all-purpose flour", "nutella", "Biscoff", "biscuits"]
presence_nuts = ["pistachio", "walnuts", "peanut", "hazelnut"]
presence_bakingpwd = ['baking powder']

forbidden = presence_egg + presence_gluten + presence_nuts + presence_bakingpwd
presence_flour = ["cornstarch", "corn starch", "glutinous rice flour", "black sesame flour", "mochiko flour", "sweet rice flour", "glutinous rice flour (mochiko flour)"]


In [78]:
name = "bread flour"
check = list(filter(lambda x: x in name, forbidden))
if len(check) > 0:
    forbid = True
else:
    forbid = False

forbid

True

In [64]:
# only include the flour existence and then its ready to rumble!! 

def check_cons_rating(ingredients):
    cons = 0 
    mean_rating = []
    forbid = 1
    for ingredient in ingredients:
        name = ingredient['ingredient']
        mean_rating.append(ingredient['rating'])
        for char in name:
           if char.lower()not in 'aeiou' and char.isalpha():
              cons += 1
        check = list(filter(lambda x: x in name, forbidden))
        if check:
            forbid = 0
    average_rating = stat.mean(mean_rating)
    return cons, average_rating, forbid

def check_forbidden(forbidden_list, ingredients):
    pass

def evaluate_recipes(recipes):
    """Evaluate the fitness of each recipe based on the average ingredient rating."""
    for r in recipes:
        # valid_ratings = [ingredient['rating'] for ingredient in r['ingredients'] if isinstance(ingredient['rating'], (int, float))]
        # if len(valid_ratings) > 0:  # Ensure there are valid ratings to avoid division by zero
        # else:
        #     print("no valid")
        #     r['fitness'] = 0 + 0.9 * check_consonants(r['ingredients']) # Default fitness for recipes with no valid ratings

        
        cons, rating, forbid = check_cons_rating(r['ingredients'])

        r['fitness'] = (0.1 * rating + 0.9 * cons)*forbid

        


Use this function to evaluate the initial population.

In [63]:
# Evaluate the fitness of each recipe in the dataset
evaluate_recipes(recipes)

# Check the recipes with their fitness scores
for recipe in recipes:
    print(f"{recipe['name']} - Fitness: {recipe['fitness']:.2f}")

evaluate_recipes(population)
population = sorted(population, reverse = True, key = lambda r: r['fitness'])

Strawberry Mochi - Fitness: 76.97
Fresh Mango - Fitness: 51.80
Applesauce Mochi - Fitness: 68.84
Sweet Potato - Fitness: 63.50
Raspberry Chocolate - Fitness: 60.80
Banana Chocolate - Fitness: 59.77
Tofu - Fitness: 77.86
Pumpkin - Fitness: 84.18
Savory Sweet Corn Mochi - Fitness: 52.60
Pistachio Butter - Fitness: 61.70
Black Sesame - Fitness: 77.00
Purple Sweet Potato Mochi - Fitness: 87.79
Green Tea / Matcha (with a Twist - Fitness: 69.80
Almond Milk - Fitness: 41.00
Crunchy Peanut Butter - Fitness: 68.00
Blueberry Mochi Ice Cream - Fitness: 70.70
Mugwort Mochi - Fitness: 73.38
Pandan Mochi - Fitness: 55.40
Black Sesame Mochi Muffins - Fitness: 68.90
Mango Mochi - Fitness: 25.58
Matcha Mochi Waffles - Fitness: 51.63
Ube Baked Mochi Donuts - Fitness: 67.85
Potato Mochi - Fitness: 28.23
Chocolate Mochi - Fitness: 57.20
Ube Mochi Muffins - Fitness: 55.25
Biscoff Baked Mochi Donuts - Fitness: 74.10
Nutella Mochi - Fitness: 27.50
Chocolate Mochi Cupcakes - Fitness: 98.53
Matcha Mochi Muffin

## Selecting recipes

Now we will choose a method for selecting recipes.

E.g., if we stick with the roulette wheel that is used in the example it would look like this:

In [ ]:
def select_recipe(recipes_copy):
  sum_fitness = sum([recipe['fitness'] for recipe in recipes_copy])
  f = random.randint(0, sum_fitness)
  for recipe in recipes_copy:
    if f < recipe['fitness']:
      return recipe
    f -= recipe['fitness']
  return recipes_copy[-1]

## Crossover and mutations

How do we choose what we want to use for crossover?

What mutations do we want to use? We can choose for example, increasing/decreasing the amount of an ingredient, substituting, adding or removing ingredients.
If we choose to substitute an ingredient, we want it to be substituted with the same type of ingredients (e.g, wets with wets, dries with dries).

In [ ]:
recipe_number = 1

def crossover_recipes(r1, r2):
  global recipe_number
  p1 = random.randint(1, len(r1['ingredients'])-1)
  p2 = random.randint(1, len(r2['ingredients'])-1)
  r1a = r1['ingredients'][0:p1]
  r2b = r2['ingredients'][p2:-1]
  r = dict()
  r['name'] = "recipe {}".format(recipe_number)
  recipe_number += 1
  r['ingredients'] = r1a + r2b
  return r

In [ ]:
def normalise_recipe(r):
  unique_ingredients = dict()
  for i in r['ingredients']:
    if i['ingredient'] in unique_ingredients:
      n = unique_ingredients[i['ingredient']]
      n['amount'] += i['amount']
    else:
      unique_ingredients[i['ingredient']] = i.copy()
  r['ingredients'] = list(unique_ingredients.values())

  sum_amounts = sum([i['amount'] for i in r['ingredients']])
  scale = 1000 / sum_amounts
  for i in r['ingredients']:
    i['amount'] = max(1, math.floor(i['amount'] * scale))